## Import Required Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Loading the Dataset


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shashanknecrothapa/ames-housing-dataset", path="AmesHousing.csv")

# load as pandas dataframe
df = pd.read_csv(path)

In [ ]:
# Dataset shape
df.shape

In [ ]:
# Dataset sample
df.head()


In [ ]:
# Columns
df.columns

## Exploratory Analysis Before Cleaning

In [ ]:
# Info
df.info()

In [ ]:
# Are any there missing values?
df.isna().any()

In [ ]:
# How many missing values?
df.isna().sum()

In [ ]:
# Sum of missing values per column
missing_values = df.isna().sum()

# Total number of rows
total_rows = len(df)

# Proportion of missing values per column
missing_value_proportion = missing_values / total_rows

print(missing_value_proportion)